In [1]:
import tensorflow as tf

/Users/fangche@cisco.com/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/fangche@cisco.com/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/fangche@cisco.com/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/fangche@cisco.com/Lib

In [4]:
shape_img = (100,100,3)
n_hidden_1, n_hidden_2, n_hidden_3 = 16, 8, 8
convkernel = (3, 3)  # convolution kernel
poolkernel = (2, 2)  # pooling kernel

input = tf.keras.layers.Input(shape=shape_img)
x = tf.keras.layers.Conv2D(n_hidden_1, convkernel, activation='relu', padding='same')(input)
x = tf.keras.layers.MaxPooling2D(poolkernel, padding='same')(x)
x = tf.keras.layers.Conv2D(n_hidden_2, convkernel, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D(poolkernel, padding='same')(x)
x = tf.keras.layers.Conv2D(n_hidden_3, convkernel, activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D(poolkernel, padding='same')(x)

x = tf.keras.layers.Conv2D(n_hidden_3, convkernel, activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D(poolkernel)(x)
x = tf.keras.layers.Conv2D(n_hidden_2, convkernel, activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D(poolkernel)(x)
x = tf.keras.layers.Conv2D(n_hidden_1, convkernel, activation='relu')(x)
x = tf.keras.layers.UpSampling2D(poolkernel)(x)
decoded = tf.keras.layers.Conv2D(shape_img[2], convkernel, activation='sigmoid', padding='same')(x)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
autoencoder = tf.keras.Model(input, decoded)

In [9]:
encoder = tf.keras.Model(input, encoded)  # set encoder
output_encoder_shape = encoder.layers[-1].output_shape[1:] #
output_encoder_shape[1:]


(13, 8)

In [14]:
decoded_input = tf.keras.Input(shape=output_encoder_shape)
decoded_output = autoencoder.layers[-7](decoded_input)  # Conv2D
decoded_output = autoencoder.layers[-6](decoded_output)  # UpSampling2D
decoded_output = autoencoder.layers[-5](decoded_output)  # Conv2D
decoded_output = autoencoder.layers[-4](decoded_output)  # UpSampling2D
decoded_output = autoencoder.layers[-3](decoded_output)  # Conv2D
decoded_output = autoencoder.layers[-2](decoded_output)  # UpSampling2D
decoded_output = autoencoder.layers[-1](decoded_output)  # Conv2D

(None, 13, 13, 8)


In [24]:
for i in range(7):
    print(autoencoder.layers[i-7].output_shape)

(None, 13, 13, 8)
(None, 26, 26, 8)
(None, 26, 26, 8)
(None, 52, 52, 8)
(None, 50, 50, 16)
(None, 100, 100, 16)
(None, 100, 100, 3)


In [25]:
autoencoder.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 8)         584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 8)         0     